In [1]:
import os
import pickle
import argparse
import numpy as np
from model import Options, Seq2SeqAttn


In [2]:
# Parse the command line arguments.
save_dir = '/Users/yan/Documents/document/EPFL/MA2/semesterprj/code/seq2seq_attn/affect-rich/input/'
output_dir = '/Users/yan/Documents/document/EPFL/MA2/semesterprj/code/seq2seq_attn/affect-rich/output/'

parser = argparse.ArgumentParser()
parser.add_argument('--data_path', type = str, default = save_dir,
                    help = 'the directory to the data')

parser.add_argument('--word_embeddings_path', type = str, default = save_dir+'word_embeddings.npy',
                    help = 'the directory to the pre-trained word embeddings')
parser.add_argument('--VAD_path', type = str, default = save_dir+'VAD.npy',
                    help = 'the directory to VAD')
parser.add_argument('--tf_path', type = str, default = save_dir+'tf.npy',
                    help = 'the directory to term frequency')
parser.add_argument('--VAD_loss_path', type = str, default = save_dir+'VAD_loss.npy',
                    help = 'the directory to VAD loss for each word')
parser.add_argument('--ti_path', type = str, default = save_dir+'mu_li.npy',
                    help = 'the directory to term importance')

parser.add_argument('--num_epochs', type = int, default = 3,
                    help = 'the number of epochs to train the data')
parser.add_argument('--batch_size', type = int, default = 64,
                    help = 'the batch size')
parser.add_argument('--learning_rate', type = float, default = 0.001,
                    help = 'the learning rate')
parser.add_argument('--beam_width', type = int, default = 32,
                    help = 'the beam width when decoding')
parser.add_argument('--word_embed_size', type = int, default = 300,
                    help = 'the size of word embeddings')
parser.add_argument('--n_hidden_units_enc', type = int, default = 256,
                    help = 'the number of hidden units of encoder')
parser.add_argument('--n_hidden_units_dec', type = int, default = 256,
                    help = 'the number of hidden units of decoder')
# ? attn_depth
parser.add_argument('--attn_depth', type = int, default = 128,
                    help = 'attention depth')

parser.add_argument('--restore_path_TS', type = str, default = output_dir+'model_dailydialog_rf_noVAD/model_TS',
                    help = 'the path to restore the trained model')
parser.add_argument('--save_path_TS', type = str, default = output_dir+'/model_dailydialog_rf_noVAD/model_TS',
                    help = 'the path to save the trained model to')

parser.add_argument('--restore_path_ST', type = str, default = output_dir+'model_dailydialog_rf_noVAD/model_ST',
                    help = 'the path to restore the trained model')
parser.add_argument('--save_path_ST', type = str, default = output_dir+'/model_dailydialog_rf_noVAD/model_ST',
                    help = 'the path to save the trained model to')

parser.add_argument('--restore_epoch', type = int, default = 0,
                    help = 'the epoch to restore')

# args = parser.parse_args()
args, unknown = parser.parse_known_args()


In [3]:
def read_data(data_path):
    def load_np_files(path):
        my_set = {}
        my_set['enc_input'] = np.load(os.path.join(path, 'enc_input.npy'))
        my_set['dec_input'] = np.load(os.path.join(path, 'dec_input.npy'))
        my_set['target'] = np.load(os.path.join(path, 'target.npy'))
        my_set['enc_input_len'] = np.load(os.path.join(path, 'enc_input_len.npy'))
        my_set['dec_input_len'] = np.load(os.path.join(path, 'dec_input_len.npy'))
        # to check if or not to complete the last batch
        idx = np.arange(my_set['dec_input'].shape[0])
        left_samples = idx[-1]%args.batch_size
        if left_samples:
            last_batch_idx = np.random.randint(0,idx[-1]-left_samples,size = args.batch_size - left_samples - 1)
            idx = np.concatenate([idx,last_batch_idx])
            
            my_set['enc_input'] = my_set['enc_input'][idx]
            my_set['dec_input'] = my_set['dec_input'][idx]
            my_set['target'] = my_set['target'][idx]
            my_set['enc_input_len'] = my_set['enc_input_len'][idx]
            my_set['dec_input_len'] = my_set['dec_input_len'][idx]
        return my_set
    train_set = load_np_files(os.path.join(data_path, 'train'))
    valid_set = load_np_files(os.path.join(data_path, 'validation'))
    
    with open(os.path.join(data_path, 'token2id.pickle'), 'rb') as file:
        token2id = pickle.load(file)
    with open(os.path.join(data_path, 'id2token.pickle'), 'rb') as file:
        id2token = pickle.load(file)

    return train_set, valid_set, token2id,id2token

---
Train model maximizing P(T|S)

In [4]:
train_set, valid_set, token2id,id2token = read_data(args.data_path)

In [5]:
train_set['enc_input'].shape

(46400, 20)

In [6]:
if __name__ == '__main__':
    train_set, valid_set, token2id,id2token = read_data(args.data_path)
#     train_set['enc_input'] = train_set['enc_input'][:128,]
    
    max_uttr_len_enc = train_set['enc_input'].shape[1]
    max_uttr_len_dec = train_set['dec_input'].shape[1]

    word_embeddings = np.load(args.word_embeddings_path)
    VAD = np.load(args.VAD_path)
    termfreq = np.load(args.ti_path) # term importance
    termfreq = termfreq.reshape(-1,1)
    VAD_loss = np.load(args.VAD_loss_path)
    VAD_loss = VAD_loss.reshape(-1,1)
    
    options = Options(mode = 'TRAIN',
                      VAD_mode = False,
                      num_epochs = args.num_epochs,
                      batch_size = args.batch_size,
                      learning_rate = args.learning_rate,
                      beam_width = args.beam_width,
                      corpus_size = len(token2id),
                      max_uttr_len_enc = max_uttr_len_enc,
                      max_uttr_len_dec = max_uttr_len_dec,
                      go_index = token2id['<go>'],
                      eos_index = token2id['<eos>'],
                      word_embed_size = args.word_embed_size,
                      n_hidden_units_enc = args.n_hidden_units_enc,
                      n_hidden_units_dec = args.n_hidden_units_dec,
                      attn_depth = args.attn_depth,
                      word_embeddings = word_embeddings)
    model_TS = Seq2SeqAttn(options)

    for var in model_TS.tvars:
        print(var.name)

    if args.restore_epoch > 0:
        model_TS.restore(os.path.join(args.restore_path_TS, 'model_epoch_{:03d}.ckpt'.format(args.restore_epoch)))
    else:
        model_TS.init_tf_vars()
    model_TS.train(train_set, VAD,termfreq, VAD_loss,args.save_path_TS, args.restore_epoch, valid_set)

Building the TensorFlow graph...
embedding/embedding:0
encoding/rnn/gru_cell/gates/kernel:0
encoding/rnn/gru_cell/gates/bias:0
encoding/rnn/gru_cell/candidate/kernel:0
encoding/rnn/gru_cell/candidate/bias:0
decoding/memory_layer/kernel:0
decoding/attention_v:0
decoding/my_bahdanau_attention/query_layer/kernel:0
decoding/attention_wrapper/gru_cell/gates/kernel:0
decoding/attention_wrapper/gru_cell/gates/bias:0
decoding/attention_wrapper/gru_cell/candidate/kernel:0
decoding/attention_wrapper/gru_cell/candidate/bias:0
decoding/dense/kernel:0
decoding/dense/bias:0
TensorFlow variables initialized.
Start to train the model...
Epoch 001/003, valid ppl = None, batch 0001/0725, train loss = 9.997103691101074
Epoch 001/003, valid ppl = None, batch 0002/0725, train loss = 9.832530975341797
Epoch 001/003, valid ppl = None, batch 0003/0725, train loss = 9.66786003112793
Epoch 001/003, valid ppl = None, batch 0004/0725, train loss = 9.470152854919434
Epoch 001/003, valid ppl = None, batch 0005/0725

Epoch 001/003, valid ppl = None, batch 0095/0725, train loss = 4.860218524932861
Epoch 001/003, valid ppl = None, batch 0096/0725, train loss = 5.312703609466553
Epoch 001/003, valid ppl = None, batch 0097/0725, train loss = 4.865604877471924
Epoch 001/003, valid ppl = None, batch 0098/0725, train loss = 4.972722053527832
Epoch 001/003, valid ppl = None, batch 0099/0725, train loss = 5.010200500488281
Epoch 001/003, valid ppl = None, batch 0100/0725, train loss = 4.911144256591797
Epoch 001/003, valid ppl = None, batch 0101/0725, train loss = 5.053136348724365
Epoch 001/003, valid ppl = None, batch 0102/0725, train loss = 4.901992321014404
Epoch 001/003, valid ppl = None, batch 0103/0725, train loss = 4.73117733001709
Epoch 001/003, valid ppl = None, batch 0104/0725, train loss = 4.737037181854248
Epoch 001/003, valid ppl = None, batch 0105/0725, train loss = 4.713125228881836
Epoch 001/003, valid ppl = None, batch 0106/0725, train loss = 4.852753162384033
Epoch 001/003, valid ppl = No

Epoch 001/003, valid ppl = None, batch 0197/0725, train loss = 4.880639553070068
Epoch 001/003, valid ppl = None, batch 0198/0725, train loss = 4.5918498039245605
Epoch 001/003, valid ppl = None, batch 0199/0725, train loss = 4.538132667541504
Epoch 001/003, valid ppl = None, batch 0200/0725, train loss = 4.615043640136719
Epoch 001/003, valid ppl = None, batch 0201/0725, train loss = 4.962582111358643
Epoch 001/003, valid ppl = None, batch 0202/0725, train loss = 4.765137672424316
Epoch 001/003, valid ppl = None, batch 0203/0725, train loss = 4.604966163635254
Epoch 001/003, valid ppl = None, batch 0204/0725, train loss = 4.798954010009766
Epoch 001/003, valid ppl = None, batch 0205/0725, train loss = 4.4516377449035645
Epoch 001/003, valid ppl = None, batch 0206/0725, train loss = 4.682681083679199
Epoch 001/003, valid ppl = None, batch 0207/0725, train loss = 4.773443698883057
Epoch 001/003, valid ppl = None, batch 0208/0725, train loss = 4.884274005889893
Epoch 001/003, valid ppl =

Epoch 001/003, valid ppl = None, batch 0299/0725, train loss = 4.177657127380371
Epoch 001/003, valid ppl = None, batch 0300/0725, train loss = 4.489805698394775
Epoch 001/003, valid ppl = None, batch 0301/0725, train loss = 4.487781524658203
Epoch 001/003, valid ppl = None, batch 0302/0725, train loss = 4.177856922149658
Epoch 001/003, valid ppl = None, batch 0303/0725, train loss = 4.5559844970703125
Epoch 001/003, valid ppl = None, batch 0304/0725, train loss = 4.410938262939453
Epoch 001/003, valid ppl = None, batch 0305/0725, train loss = 4.434031009674072
Epoch 001/003, valid ppl = None, batch 0306/0725, train loss = 4.398378372192383
Epoch 001/003, valid ppl = None, batch 0307/0725, train loss = 4.572534561157227
Epoch 001/003, valid ppl = None, batch 0308/0725, train loss = 4.58293342590332
Epoch 001/003, valid ppl = None, batch 0309/0725, train loss = 4.574490547180176
Epoch 001/003, valid ppl = None, batch 0310/0725, train loss = 4.631105422973633
Epoch 001/003, valid ppl = N

Epoch 001/003, valid ppl = None, batch 0401/0725, train loss = 4.294539451599121
Epoch 001/003, valid ppl = None, batch 0402/0725, train loss = 4.430874347686768
Epoch 001/003, valid ppl = None, batch 0403/0725, train loss = 4.599501609802246
Epoch 001/003, valid ppl = None, batch 0404/0725, train loss = 4.23789119720459
Epoch 001/003, valid ppl = None, batch 0405/0725, train loss = 4.485577583312988
Epoch 001/003, valid ppl = None, batch 0406/0725, train loss = 4.407413959503174
Epoch 001/003, valid ppl = None, batch 0407/0725, train loss = 4.384725093841553
Epoch 001/003, valid ppl = None, batch 0408/0725, train loss = 4.190430641174316
Epoch 001/003, valid ppl = None, batch 0409/0725, train loss = 4.640187740325928
Epoch 001/003, valid ppl = None, batch 0410/0725, train loss = 4.180296897888184
Epoch 001/003, valid ppl = None, batch 0411/0725, train loss = 4.167431354522705
Epoch 001/003, valid ppl = None, batch 0412/0725, train loss = 4.247748851776123
Epoch 001/003, valid ppl = No

...

Epoch 002/003, valid ppl = 63.3399736056754, batch 0159/0725, train loss = 3.904268503189087
Epoch 002/003, valid ppl = 63.3399736056754, batch 0160/0725, train loss = 3.9054453372955322
Epoch 002/003, valid ppl = 63.3399736056754, batch 0161/0725, train loss = 3.865385055541992
Epoch 002/003, valid ppl = 63.3399736056754, batch 0162/0725, train loss = 3.9950742721557617
Epoch 002/003, valid ppl = 63.3399736056754, batch 0163/0725, train loss = 4.017649173736572
Epoch 002/003, valid ppl = 63.3399736056754, batch 0164/0725, train loss = 3.6728906631469727
Epoch 002/003, valid ppl = 63.3399736056754, batch 0165/0725, train loss = 3.9152116775512695
Epoch 002/003, valid ppl = 63.3399736056754, batch 0166/0725, train loss = 3.8423895835876465
Epoch 002/003, valid ppl = 63.3399736056754, batch 0167/0725, train loss = 3.857224941253662
Epoch 002/003, valid ppl = 63.3399736056754, batch 0168/0725, train loss = 3.9080278873443604
Epoch 002/003, valid ppl = 63.3399736056754, batch 0169/0725, tr

Epoch 002/003, valid ppl = 63.3399736056754, batch 0247/0725, train loss = 3.821248769760132
Epoch 002/003, valid ppl = 63.3399736056754, batch 0248/0725, train loss = 3.890195608139038
Epoch 002/003, valid ppl = 63.3399736056754, batch 0249/0725, train loss = 3.747215509414673
Epoch 002/003, valid ppl = 63.3399736056754, batch 0250/0725, train loss = 3.905595541000366
Epoch 002/003, valid ppl = 63.3399736056754, batch 0251/0725, train loss = 3.6937172412872314
Epoch 002/003, valid ppl = 63.3399736056754, batch 0252/0725, train loss = 3.89815354347229
Epoch 002/003, valid ppl = 63.3399736056754, batch 0253/0725, train loss = 4.024538516998291
Epoch 002/003, valid ppl = 63.3399736056754, batch 0254/0725, train loss = 3.8715977668762207
Epoch 002/003, valid ppl = 63.3399736056754, batch 0255/0725, train loss = 4.019964218139648
Epoch 002/003, valid ppl = 63.3399736056754, batch 0256/0725, train loss = 4.136954307556152
Epoch 002/003, valid ppl = 63.3399736056754, batch 0257/0725, train l

Epoch 002/003, valid ppl = 63.3399736056754, batch 0335/0725, train loss = 4.052115440368652
Epoch 002/003, valid ppl = 63.3399736056754, batch 0336/0725, train loss = 3.778634548187256
Epoch 002/003, valid ppl = 63.3399736056754, batch 0337/0725, train loss = 3.4545364379882812
Epoch 002/003, valid ppl = 63.3399736056754, batch 0338/0725, train loss = 3.750166893005371
Epoch 002/003, valid ppl = 63.3399736056754, batch 0339/0725, train loss = 4.0358662605285645
Epoch 002/003, valid ppl = 63.3399736056754, batch 0340/0725, train loss = 3.936466693878174
Epoch 002/003, valid ppl = 63.3399736056754, batch 0341/0725, train loss = 3.6995370388031006
Epoch 002/003, valid ppl = 63.3399736056754, batch 0342/0725, train loss = 3.694521903991699
Epoch 002/003, valid ppl = 63.3399736056754, batch 0343/0725, train loss = 3.867457866668701
Epoch 002/003, valid ppl = 63.3399736056754, batch 0344/0725, train loss = 3.7004921436309814
Epoch 002/003, valid ppl = 63.3399736056754, batch 0345/0725, trai

Epoch 002/003, valid ppl = 63.3399736056754, batch 0423/0725, train loss = 3.7162914276123047
Epoch 002/003, valid ppl = 63.3399736056754, batch 0424/0725, train loss = 3.887932538986206
Epoch 002/003, valid ppl = 63.3399736056754, batch 0425/0725, train loss = 3.770714521408081
Epoch 002/003, valid ppl = 63.3399736056754, batch 0426/0725, train loss = 3.7122435569763184
Epoch 002/003, valid ppl = 63.3399736056754, batch 0427/0725, train loss = 3.833315134048462
Epoch 002/003, valid ppl = 63.3399736056754, batch 0428/0725, train loss = 3.6321356296539307
Epoch 002/003, valid ppl = 63.3399736056754, batch 0429/0725, train loss = 3.685088872909546
Epoch 002/003, valid ppl = 63.3399736056754, batch 0430/0725, train loss = 3.804158926010132
Epoch 002/003, valid ppl = 63.3399736056754, batch 0431/0725, train loss = 3.803469657897949
Epoch 002/003, valid ppl = 63.3399736056754, batch 0432/0725, train loss = 3.7077527046203613
Epoch 002/003, valid ppl = 63.3399736056754, batch 0433/0725, trai

KeyboardInterrupt: 

---
Train model P(S|T)

In [7]:
def revert(myset):
    enc_input = myset['dec_input'][:,1:]
    dec_input =  np.insert(myset['enc_input'], 0, token2id['<go>'], axis=1) # add <go> in the beginning of decoder

    target = np.insert(myset['enc_input'], -1, 0, axis=1) 
    tmp_idx = [np.where(s==0)[0][0] for s in target] 
    target[np.arange(target.shape[0]),tmp_idx] = token2id['<eos>'] # add <eos> at the end of decoder
    
    newset = {}
    
    newset['enc_input'] = enc_input
    newset['dec_input'] = dec_input
    newset['target'] = target
    newset['enc_input_len'] = myset['dec_input_len']
    newset['dec_input_len'] = myset['enc_input_len']
    return newset

In [8]:
if __name__ == '__main__':
    train_set, valid_set, token2id,id2token = read_data(args.data_path)
    train_set = revert(train_set)
    valid_set = revert(valid_set)
#     train_set['enc_input'] = train_set['enc_input'][:128,]
    
    max_uttr_len_enc = train_set['enc_input'].shape[1]
    max_uttr_len_dec = train_set['dec_input'].shape[1]

    word_embeddings = np.load(args.word_embeddings_path)
    VAD = np.load(args.VAD_path)
    termfreq = np.load(args.ti_path) # term importance
    termfreq = termfreq.reshape(-1,1)
    VAD_loss = np.load(args.VAD_loss_path)
    VAD_loss = VAD_loss.reshape(-1,1)
    
    options = Options(mode = 'TRAIN',
                      VAD_mode = 'FALSE',
                      num_epochs = args.num_epochs,
                      batch_size = args.batch_size,
                      learning_rate = args.learning_rate,
                      beam_width = args.beam_width,
                      corpus_size = len(token2id),
                      max_uttr_len_enc = max_uttr_len_enc,
                      max_uttr_len_dec = max_uttr_len_dec,
                      go_index = token2id['<go>'],
                      eos_index = token2id['<eos>'],
                      word_embed_size = args.word_embed_size,
                      n_hidden_units_enc = args.n_hidden_units_enc,
                      n_hidden_units_dec = args.n_hidden_units_dec,
                      attn_depth = args.attn_depth,
                      word_embeddings = word_embeddings)
    model_ST = Seq2SeqAttn(options)

    for var in model_ST.tvars:
        print(var.name)

    if args.restore_epoch > 0:
        model_ST.restore(os.path.join(args.restore_path_ST, 'model_epoch_{:03d}.ckpt'.format(args.restore_epoch)))
    else:
        model_ST.init_tf_vars()
    model_ST.train(train_set, VAD,termfreq, VAD_loss,args.save_path_ST, args.restore_epoch, valid_set)

Building the TensorFlow graph...
embedding/embedding:0
encoding/rnn/gru_cell/gates/kernel:0
encoding/rnn/gru_cell/gates/bias:0
encoding/rnn/gru_cell/candidate/kernel:0
encoding/rnn/gru_cell/candidate/bias:0
decoding/memory_layer/kernel:0
decoding/attention_v:0
decoding/my_bahdanau_attention/query_layer/kernel:0
decoding/my_bahdanau_attention/attention_Wb/kernel:0
decoding/attention_wrapper/gru_cell/gates/kernel:0
decoding/attention_wrapper/gru_cell/gates/bias:0
decoding/attention_wrapper/gru_cell/candidate/kernel:0
decoding/attention_wrapper/gru_cell/candidate/bias:0
decoding/dense/kernel:0
decoding/dense/bias:0
TensorFlow variables initialized.
Start to train the model...
Epoch 001/003, valid ppl = None, batch 0001/0725, train loss = 9.990707397460938
Epoch 001/003, valid ppl = None, batch 0002/0725, train loss = 9.855420112609863
Epoch 001/003, valid ppl = None, batch 0003/0725, train loss = 9.725510597229004
Epoch 001/003, valid ppl = None, batch 0004/0725, train loss = 9.5158042907

Epoch 001/003, valid ppl = None, batch 0094/0725, train loss = 4.883930206298828
Epoch 001/003, valid ppl = None, batch 0095/0725, train loss = 5.187080383300781
Epoch 001/003, valid ppl = None, batch 0096/0725, train loss = 5.111042022705078
Epoch 001/003, valid ppl = None, batch 0097/0725, train loss = 5.306817054748535
Epoch 001/003, valid ppl = None, batch 0098/0725, train loss = 5.258883476257324
Epoch 001/003, valid ppl = None, batch 0099/0725, train loss = 5.0688982009887695
Epoch 001/003, valid ppl = None, batch 0100/0725, train loss = 5.270203113555908
Epoch 001/003, valid ppl = None, batch 0101/0725, train loss = 4.951502799987793
Epoch 001/003, valid ppl = None, batch 0102/0725, train loss = 4.980239391326904
Epoch 001/003, valid ppl = None, batch 0103/0725, train loss = 5.097230911254883
Epoch 001/003, valid ppl = None, batch 0104/0725, train loss = 5.236363887786865
Epoch 001/003, valid ppl = None, batch 0105/0725, train loss = 5.241433620452881
Epoch 001/003, valid ppl = 

Epoch 001/003, valid ppl = None, batch 0196/0725, train loss = 4.815104961395264
Epoch 001/003, valid ppl = None, batch 0197/0725, train loss = 4.779452800750732
Epoch 001/003, valid ppl = None, batch 0198/0725, train loss = 4.60347318649292
Epoch 001/003, valid ppl = None, batch 0199/0725, train loss = 4.8945183753967285
Epoch 001/003, valid ppl = None, batch 0200/0725, train loss = 4.635896682739258
Epoch 001/003, valid ppl = None, batch 0201/0725, train loss = 4.866043567657471
Epoch 001/003, valid ppl = None, batch 0202/0725, train loss = 4.663118362426758
Epoch 001/003, valid ppl = None, batch 0203/0725, train loss = 4.947104454040527
Epoch 001/003, valid ppl = None, batch 0204/0725, train loss = 4.838286399841309
Epoch 001/003, valid ppl = None, batch 0205/0725, train loss = 4.4580078125
Epoch 001/003, valid ppl = None, batch 0206/0725, train loss = 4.3888726234436035
Epoch 001/003, valid ppl = None, batch 0207/0725, train loss = 4.5985822677612305
Epoch 001/003, valid ppl = None

Epoch 001/003, valid ppl = None, batch 0298/0725, train loss = 4.119123935699463
Epoch 001/003, valid ppl = None, batch 0299/0725, train loss = 4.507351875305176
Epoch 001/003, valid ppl = None, batch 0300/0725, train loss = 4.299500942230225
Epoch 001/003, valid ppl = None, batch 0301/0725, train loss = 4.645863056182861
Epoch 001/003, valid ppl = None, batch 0302/0725, train loss = 4.926673412322998
Epoch 001/003, valid ppl = None, batch 0303/0725, train loss = 4.456812381744385
Epoch 001/003, valid ppl = None, batch 0304/0725, train loss = 4.607220649719238
Epoch 001/003, valid ppl = None, batch 0305/0725, train loss = 4.775251388549805
Epoch 001/003, valid ppl = None, batch 0306/0725, train loss = 4.679826736450195
Epoch 001/003, valid ppl = None, batch 0307/0725, train loss = 4.278611183166504
Epoch 001/003, valid ppl = None, batch 0308/0725, train loss = 4.334272861480713
Epoch 001/003, valid ppl = None, batch 0309/0725, train loss = 4.471628189086914
Epoch 001/003, valid ppl = N

Epoch 001/003, valid ppl = None, batch 0400/0725, train loss = 4.366425514221191
Epoch 001/003, valid ppl = None, batch 0401/0725, train loss = 4.580059051513672
Epoch 001/003, valid ppl = None, batch 0402/0725, train loss = 4.464258193969727
Epoch 001/003, valid ppl = None, batch 0403/0725, train loss = 4.154728412628174
Epoch 001/003, valid ppl = None, batch 0404/0725, train loss = 4.295827865600586
Epoch 001/003, valid ppl = None, batch 0405/0725, train loss = 4.284739971160889
Epoch 001/003, valid ppl = None, batch 0406/0725, train loss = 4.624973773956299
Epoch 001/003, valid ppl = None, batch 0407/0725, train loss = 4.4171319007873535
Epoch 001/003, valid ppl = None, batch 0408/0725, train loss = 4.193425178527832
Epoch 001/003, valid ppl = None, batch 0409/0725, train loss = 4.662446975708008
Epoch 001/003, valid ppl = None, batch 0410/0725, train loss = 4.421490669250488
Epoch 001/003, valid ppl = None, batch 0411/0725, train loss = 4.228417873382568
Epoch 001/003, valid ppl = 

...

Epoch 003/003, valid ppl = 43.60350735564386, batch 0379/0725, train loss = 3.3145933151245117
Epoch 003/003, valid ppl = 43.60350735564386, batch 0380/0725, train loss = 3.4441065788269043
Epoch 003/003, valid ppl = 43.60350735564386, batch 0381/0725, train loss = 3.3188514709472656
Epoch 003/003, valid ppl = 43.60350735564386, batch 0382/0725, train loss = 3.2015891075134277
Epoch 003/003, valid ppl = 43.60350735564386, batch 0383/0725, train loss = 3.0426316261291504
Epoch 003/003, valid ppl = 43.60350735564386, batch 0384/0725, train loss = 3.1288721561431885
Epoch 003/003, valid ppl = 43.60350735564386, batch 0385/0725, train loss = 3.166163444519043
Epoch 003/003, valid ppl = 43.60350735564386, batch 0386/0725, train loss = 3.037213087081909
Epoch 003/003, valid ppl = 43.60350735564386, batch 0387/0725, train loss = 3.1841118335723877
Epoch 003/003, valid ppl = 43.60350735564386, batch 0388/0725, train loss = 3.234384536743164
Epoch 003/003, valid ppl = 43.60350735564386, batch 0

Epoch 003/003, valid ppl = 43.60350735564386, batch 0466/0725, train loss = 3.5140397548675537
Epoch 003/003, valid ppl = 43.60350735564386, batch 0467/0725, train loss = 3.065960645675659
Epoch 003/003, valid ppl = 43.60350735564386, batch 0468/0725, train loss = 3.2700235843658447
Epoch 003/003, valid ppl = 43.60350735564386, batch 0469/0725, train loss = 3.018458366394043
Epoch 003/003, valid ppl = 43.60350735564386, batch 0470/0725, train loss = 2.9374191761016846
Epoch 003/003, valid ppl = 43.60350735564386, batch 0471/0725, train loss = 3.0517940521240234
Epoch 003/003, valid ppl = 43.60350735564386, batch 0472/0725, train loss = 3.108652114868164
Epoch 003/003, valid ppl = 43.60350735564386, batch 0473/0725, train loss = 3.1740384101867676
Epoch 003/003, valid ppl = 43.60350735564386, batch 0474/0725, train loss = 3.06602144241333
Epoch 003/003, valid ppl = 43.60350735564386, batch 0475/0725, train loss = 3.1689038276672363
Epoch 003/003, valid ppl = 43.60350735564386, batch 047

Epoch 003/003, valid ppl = 43.60350735564386, batch 0553/0725, train loss = 2.980897903442383
Epoch 003/003, valid ppl = 43.60350735564386, batch 0554/0725, train loss = 3.0589253902435303
Epoch 003/003, valid ppl = 43.60350735564386, batch 0555/0725, train loss = 3.127553939819336
Epoch 003/003, valid ppl = 43.60350735564386, batch 0556/0725, train loss = 3.014857769012451
Epoch 003/003, valid ppl = 43.60350735564386, batch 0557/0725, train loss = 3.015998125076294
Epoch 003/003, valid ppl = 43.60350735564386, batch 0558/0725, train loss = 2.773847818374634
Epoch 003/003, valid ppl = 43.60350735564386, batch 0559/0725, train loss = 3.2930099964141846
Epoch 003/003, valid ppl = 43.60350735564386, batch 0560/0725, train loss = 3.3181660175323486
Epoch 003/003, valid ppl = 43.60350735564386, batch 0561/0725, train loss = 2.977242946624756
Epoch 003/003, valid ppl = 43.60350735564386, batch 0562/0725, train loss = 3.036402702331543
Epoch 003/003, valid ppl = 43.60350735564386, batch 0563/

Epoch 003/003, valid ppl = 43.60350735564386, batch 0640/0725, train loss = 3.004286527633667
Epoch 003/003, valid ppl = 43.60350735564386, batch 0641/0725, train loss = 2.9617421627044678
Epoch 003/003, valid ppl = 43.60350735564386, batch 0642/0725, train loss = 3.2117817401885986
Epoch 003/003, valid ppl = 43.60350735564386, batch 0643/0725, train loss = 3.2269175052642822
Epoch 003/003, valid ppl = 43.60350735564386, batch 0644/0725, train loss = 3.0800302028656006
Epoch 003/003, valid ppl = 43.60350735564386, batch 0645/0725, train loss = 2.9972009658813477
Epoch 003/003, valid ppl = 43.60350735564386, batch 0646/0725, train loss = 3.1627960205078125
Epoch 003/003, valid ppl = 43.60350735564386, batch 0647/0725, train loss = 2.980532646179199
Epoch 003/003, valid ppl = 43.60350735564386, batch 0648/0725, train loss = 3.012706756591797
Epoch 003/003, valid ppl = 43.60350735564386, batch 0649/0725, train loss = 3.0420706272125244
Epoch 003/003, valid ppl = 43.60350735564386, batch 0

Epoch 001, valid ppl = 60.682336799751084
Epoch 002, valid ppl = 43.60350735564386
Epoch 003, valid ppl = 34.67138074459025
